In [19]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [20]:
wfigs_train=pd.read_csv("wfigs.csv")

In [21]:
#checkinig for number of rows and columns

In [22]:
wfigs_train.shape

(7125, 105)

In [23]:
wfigs_train.head()

,OBJECTID,poly_IncidentName,poly_FeatureCategory,poly_MapMethod,poly_GISAcres,poly_CreateDate,poly_DateCurrent,poly_PolygonDateTime,poly_Acres_AutoCalc,poly_GlobalID,...,irwin_OrganizationalAssessment,irwin_StratDecisionPublishDate,irwin_GlobalID,irwin_Source,irwin_ArchivedOn,irwin_ModifiedOnDateTime_dt,irwin_CreatedOnDateTime_dt,GlobalID,SHAPE_Length,SHAPE_Area
0,127,Coleman,Wildfire Daily Fire Perimeter,Mixed Methods,562.913504,2021/05/04 22:50:38+00,2021/05/04 22:50:38+00,NaN,562.911767,{951569B8-EA6B-443B-8BF7-022AE1DFC937},...,Type 3 Incident,2020/10/20 14:14:11+00,NaN,IRWIN,NaN,2021/03/26 19:01:00+00,2020/10/18 20:02:59+00,{1E48F205-2208-4DBD-8A48-B739FDEACF9C},0.120036,2.280140e-04
1,128,Patrick,Wildfire Daily Fire Perimeter,Mixed Methods,0.151680,2021/05/04 22:51:03+00,2021/05/04 22:51:03+00,NaN,0.151680,{08D5DE2B-51A0-420A-812F-35902565E93A},...,NaN,NaN,NaN,IRWIN,NaN,2020/06/19 20:31:59+00,2020/05/01 22:15:24+00,{447213E7-190B-4E48-B60C-D0E492EFD576},0.001121,6.430000e-08
2,129,Low Top,Wildfire Daily Fire Perimeter,Auto-generated,0.300000,2021/05/04 22:51:19+00,2021/05/04 22:51:19+00,2020/08/09 04:24:22+00,2.850691,{4BBFA91F-8692-4FC6-8917-1E7C2F2678EE},...,NaN,NaN,NaN,IRWIN,NaN,2020/08/27 18:54:27+00,2020/08/08 23:47:45+00,{EC414CFA-9C17-478C-A07E-849E91501621},0.004527,1.120000e-06
3,130,Sloan 2,Wildfire Daily Fire Perimeter,Mixed Methods,44.300517,2021/05/04 22:50:52+00,2021/05/04 22:50:52+00,NaN,44.300352,{7AE9D007-FBA4-440B-972B-69791E156BE8},...,NaN,NaN,NaN,IRWIN,NaN,2021/01/27 20:56:40+00,2020/05/08 02:54:29+00,{C882C4C8-1420-4D68-9012-B24EDA77D6B7},0.032394,1.790000e-05
4,133,Bloody nose,Wildfire Daily Fire Perimeter,Mixed Methods,NaN,2021/05/04 22:50:58+00,2021/05/04 22:50:58+00,NaN,6.211367,{A3BBB888-93FB-405C-A449-EF675D72C65E},...,NaN,NaN,NaN,IRWIN,NaN,2021/01/13 18:16:55+00,2020/08/21 20:21:44+00,{01476F18-2DC0-473C-BF80-58544FA68344},0.008458,2.780000e-06


In [24]:
wfigs_train_final = wfigs_train[["OBJECTID","poly_CreateDate","poly_GISAcres","irwin_FireCause","irwin_GACC","irwin_InitialLatitude","irwin_InitialLongitude","irwin_POODispatchCenterID","irwin_POOState","irwin_PredominantFuelGroup"]]

In [25]:
column_means = wfigs_train_final.mean()
print(column_means)
column_means.shape

OBJECTID                  7925.527579
poly_GISAcres             1979.660336
irwin_InitialLatitude       40.865592
irwin_InitialLongitude    -109.575179
dtype: float64


(4,)

In [26]:
wfigs_train_final = wfigs_train_final.fillna(column_means)

In [27]:
wfigs_train_final.head()

,OBJECTID,poly_CreateDate,poly_GISAcres,irwin_FireCause,irwin_GACC,irwin_InitialLatitude,irwin_InitialLongitude,irwin_POODispatchCenterID,irwin_POOState,irwin_PredominantFuelGroup
0,127,2021/05/04 22:50:38+00,562.913504,Unknown,OSCC,36.071140,-121.450500,CALPCC,US-CA,NaN
1,128,2021/05/04 22:51:03+00,0.151680,Unknown,GBCC,39.556690,-119.558500,NVSFC,US-NV,Grass
2,129,2021/05/04 22:51:19+00,0.300000,Human,SWCC,33.293840,-110.450000,AZPHC,US-AZ,Grass-Shrub
3,130,2021/05/04 22:50:52+00,44.300517,Human,GBCC,35.875820,-115.204100,NVLIC,US-NV,Grass-Shrub
4,133,2021/05/04 22:50:58+00,1979.660336,Human,RMCC,40.865592,-109.575179,SDGPC,US-SD,Grass


In [28]:
#checking for columns headers (variable names)

In [29]:
wfigs_train.columns

Index(['OBJECTID', 'poly_IncidentName', 'poly_FeatureCategory',
       'poly_MapMethod', 'poly_GISAcres', 'poly_CreateDate',
       'poly_DateCurrent', 'poly_PolygonDateTime', 'poly_Acres_AutoCalc',
       'poly_GlobalID',
       ...
       'irwin_OrganizationalAssessment', 'irwin_StratDecisionPublishDate',
       'irwin_GlobalID', 'irwin_Source', 'irwin_ArchivedOn',
       'irwin_ModifiedOnDateTime_dt', 'irwin_CreatedOnDateTime_dt', 'GlobalID',
       'SHAPE_Length', 'SHAPE_Area'],
      dtype='object', length=105)

In [30]:
wfigs_train.head ()

,OBJECTID,poly_IncidentName,poly_FeatureCategory,poly_MapMethod,poly_GISAcres,poly_CreateDate,poly_DateCurrent,poly_PolygonDateTime,poly_Acres_AutoCalc,poly_GlobalID,...,irwin_OrganizationalAssessment,irwin_StratDecisionPublishDate,irwin_GlobalID,irwin_Source,irwin_ArchivedOn,irwin_ModifiedOnDateTime_dt,irwin_CreatedOnDateTime_dt,GlobalID,SHAPE_Length,SHAPE_Area
0,127,Coleman,Wildfire Daily Fire Perimeter,Mixed Methods,562.913504,2021/05/04 22:50:38+00,2021/05/04 22:50:38+00,NaN,562.911767,{951569B8-EA6B-443B-8BF7-022AE1DFC937},...,Type 3 Incident,2020/10/20 14:14:11+00,NaN,IRWIN,NaN,2021/03/26 19:01:00+00,2020/10/18 20:02:59+00,{1E48F205-2208-4DBD-8A48-B739FDEACF9C},0.120036,2.280140e-04
1,128,Patrick,Wildfire Daily Fire Perimeter,Mixed Methods,0.151680,2021/05/04 22:51:03+00,2021/05/04 22:51:03+00,NaN,0.151680,{08D5DE2B-51A0-420A-812F-35902565E93A},...,NaN,NaN,NaN,IRWIN,NaN,2020/06/19 20:31:59+00,2020/05/01 22:15:24+00,{447213E7-190B-4E48-B60C-D0E492EFD576},0.001121,6.430000e-08
2,129,Low Top,Wildfire Daily Fire Perimeter,Auto-generated,0.300000,2021/05/04 22:51:19+00,2021/05/04 22:51:19+00,2020/08/09 04:24:22+00,2.850691,{4BBFA91F-8692-4FC6-8917-1E7C2F2678EE},...,NaN,NaN,NaN,IRWIN,NaN,2020/08/27 18:54:27+00,2020/08/08 23:47:45+00,{EC414CFA-9C17-478C-A07E-849E91501621},0.004527,1.120000e-06
3,130,Sloan 2,Wildfire Daily Fire Perimeter,Mixed Methods,44.300517,2021/05/04 22:50:52+00,2021/05/04 22:50:52+00,NaN,44.300352,{7AE9D007-FBA4-440B-972B-69791E156BE8},...,NaN,NaN,NaN,IRWIN,NaN,2021/01/27 20:56:40+00,2020/05/08 02:54:29+00,{C882C4C8-1420-4D68-9012-B24EDA77D6B7},0.032394,1.790000e-05
4,133,Bloody nose,Wildfire Daily Fire Perimeter,Mixed Methods,NaN,2021/05/04 22:50:58+00,2021/05/04 22:50:58+00,NaN,6.211367,{A3BBB888-93FB-405C-A449-EF675D72C65E},...,NaN,NaN,NaN,IRWIN,NaN,2021/01/13 18:16:55+00,2020/08/21 20:21:44+00,{01476F18-2DC0-473C-BF80-58544FA68344},0.008458,2.780000e-06


In [31]:
#adding index field

In [32]:
wfigs_train ['index'] = pd.Series (range(0, 7125))

In [33]:
wfigs_train.head

<bound method NDFrame.head of       OBJECTID     poly_IncidentName           poly_FeatureCategory  \
0          127               Coleman  Wildfire Daily Fire Perimeter   
1          128              Patrick   Wildfire Daily Fire Perimeter   
2          129               Low Top  Wildfire Daily Fire Perimeter   
3          130               Sloan 2  Wildfire Daily Fire Perimeter   
4          133           Bloody nose  Wildfire Daily Fire Perimeter   
...        ...                   ...                            ...   
7120     12712          Ashpile Peak  Wildfire Daily Fire Perimeter   
7121     12713  Bright Angel FY21 RX                Prescribed Fire   
7122     12714          Cave Gulch 2  Wildfire Daily Fire Perimeter   
7123     12716                   S-1  Wildfire Daily Fire Perimeter   
7124     12717       Bike Trail Fire  Wildfire Daily Fire Perimeter   

      poly_MapMethod  poly_GISAcres         poly_CreateDate  \
0      Mixed Methods     562.913504  2021/05/04 22:50:

In [34]:
#checking for nulls

In [35]:
wfigs_train.isnull().sum()

OBJECTID                         0
poly_IncidentName              103
poly_FeatureCategory             0
poly_MapMethod                   5
poly_GISAcres                 1698
                              ... 
irwin_CreatedOnDateTime_dt       0
GlobalID                         0
SHAPE_Length                     0
SHAPE_Area                       0
index                            0
Length: 106, dtype: int64